In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import keras
from tensorflow import keras
from tensorflow.keras import layers
import copy
from copy import deepcopy
from keras.utils import to_categorical

In [ ]:
class Main():
  def __init__(self):
        # Constructor method
        from google.colab import drive

        #Mounting to Google Drive
        drive.mount('/content/drive')

  # def readingDataFromCSV(self):
  #       import csv
  #       import numpy as np
  #       #Reading the data from CSV
  #       inputs=[]
  #       with open('/content/drive/MyDrive/cleanDataWithNameChnagedToNumber.csv', newline='') as csvfile:
  #           reader = csv.reader(csvfile, delimiter=',')
  #           for row in reader:
  #             inputs.append(row)

  #       inputs= np.array(inputs).astype(float)


  #       #Reading Label CSV
  #       label_data=[]
  #       with open('/content/drive/MyDrive/Label.csv', newline='') as csvfile:
  #         reader = csv.reader(csvfile, delimiter=',')
  #         for row in reader:
  #           label_data.append(row)

  #       label_data = np.array(label_data).astype(float)

  #       return inputs,label_data

  # def dataSplit(self,inputs,label_data):
  def dataSplit(self):
          from sklearn.model_selection import train_test_split
          from sklearn.utils import shuffle
          import numpy as np
          
          # # Load the .npy file
          # my_array = np.load('/content/drive/MyDrive/train_data.npy')

          # # Print the shape of the array
          # print(my_array.shape)

          # Load the .npy file

          my_array = np.load('/content/drive/MyDrive/train_data.npy')
          my_array1 = np.load('/content/drive/MyDrive/y_train.npy')
          my_array2 = np.load('/content/drive/MyDrive/test_data.npy')
          my_array3 = np.load('/content/drive/MyDrive/y_test.npy')
          train_data = np.append(my_array, my_array2, axis=0)
          label_data = np.append(my_array1, my_array3, axis=0)
          train_data_copy, test_data, train_label_data_copy,test_label_data = train_test_split(train_data,label_data, train_size=40000, test_size=5714)

          print(my_array1.shape)
          print(train_label_data_copy)
                    

          # train_data_copy, test_data, train_label_data_copy,test_label_data = train_test_split(inputs,label_data, train_size=40000, test_size=5000,random_state=42)
          
          # train_data_copy, test_data= train_test_split(inputs, train_size=40000, test_size=5000)

          # train_label_data_copy, test_label_data=train_test_split(label_data, train_size=40000,test_size=5000)

          #Shuffle
          # train_data_copy, train_label_data_copy = shuffle(train_data_copy, train_label_data_copy, random_state=42)

          #Shuffle
          # test_data, test_label_data = shuffle(test_data, test_label_data, random_state=42)

          return train_data_copy,test_data,train_label_data_copy,test_label_data







In [ ]:
class Model():
  def modelDef(units):
    import tensorflow as tf

    #Define the architecture of the model
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(units, activation='relu', input_shape=(24,)),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    #Compile the mode
    model.compile(optimizer='adam', loss='binary_crossentropy')

    return model

  # Compile the model
  #def modelCompile():    
    
  #Manually Set the weights
  def ModelSetWeights(model,randomWeightsForLayer):
   model.layers[0].set_weights(randomWeightsForLayer[0])
   model.layers[1].set_weights(randomWeightsForLayer[1])

   return model

  def ModelFit(model,temp_train,temp_label):
    model.fit(temp_train, temp_label, epochs=4)
    weightsZero = model.layers[0].get_weights()
    weightsOne=model.layers[1].get_weights()
    return weightsZero,weightsOne,model

  
    

In [ ]:
class Server(Model):
  def randomWeights(self,units):
    model=Model.modelDef(units)
    randomWeightsForLayer=[]
    randomWeightsForLayerZero=[]
    randomWeightsForLayerOne=[]
    randomWeightsForLayerZero.append(model.layers[0].get_weights()[0])
    randomWeightsForLayerZero.append(model.layers[0].get_weights()[1])
    randomWeightsForLayerOne.append(model.layers[1].get_weights()[0])
    randomWeightsForLayerOne.append(model.layers[1].get_weights()[1])
    randomWeightsForLayer.append(randomWeightsForLayerZero)
    randomWeightsForLayer.append(randomWeightsForLayerOne)

    #print(model.layers[0].get_weights)
    return randomWeightsForLayer

  def dropAvg(self,weightAfterTraining,biasAfterTraining,weightAfterTrainingOne,biasAfterTrainingOne,dropClient):
    averageWeightsLayerZero=np.zeros_like(weightAfterTraining[0])


    for i in range(len(weightAfterTraining)):
      if(dropClient==i):
        continue
      averageWeightsLayerZero=np.add(averageWeightsLayerZero,weightAfterTraining[i])

    averageWeightsLayerZero=averageWeightsLayerZero/numberOfClients-1
    #print(averageWeightsLayerZero)



    
    averageWeightsLayerOne=np.zeros_like(weightAfterTrainingOne[0])

    for i in range(len(weightAfterTrainingOne)):
      if(dropClient==i):
        continue
      averageWeightsLayerOne=np.add(averageWeightsLayerOne,weightAfterTrainingOne[i])
    
    averageWeightsLayerOne=averageWeightsLayerOne/numberOfClients-1
    #print(averageWeightsLayerOne)


    
    averageBiasLayerZero=np.zeros_like(biasAfterTraining[0])

    for i in range(len(biasAfterTraining)):
      if(dropClient==i):
        continue
      averageBiasLayerZero=np.add(averageBiasLayerZero,biasAfterTraining[i])

    averageBiasLayerZero=averageBiasLayerZero/numberOfClients-1
    #print(averageBiasLayerZero)
    




    averageBiasLayerOne = np.zeros_like(biasAfterTrainingOne[0])

    for i in range(len(biasAfterTrainingOne)):
      if(dropClient==i):
        continue
      averageBiasLayerOne=np.add(averageBiasLayerOne,biasAfterTrainingOne[i])

    averageBiasLayerOne=averageBiasLayerOne/numberOfClients-1
    #print(averageBiasLayerOne)

    
    
    #print(weightAfterTraining)
    #print(biasAfterTraining)
    WeightsForServerLayerZero=[]
    WeightsForServerLayerOne=[]
    WeightsForServerLayerZero.append(averageWeightsLayerZero)
    WeightsForServerLayerZero.append(averageBiasLayerZero)
    WeightsForServerLayerOne.append(averageWeightsLayerOne)
    WeightsForServerLayerOne.append(averageBiasLayerOne)
    WeightsForServer=[]
    WeightsForServer.append(WeightsForServerLayerZero)
    WeightsForServer.append(WeightsForServerLayerOne)

    #WeightsForServer=server.calculateAvg(weightAfterTraining,biasAfterTraining,weightAfterTrainingOne,biasAfterTrainingOne)
    return WeightsForServer



  def calculateAvg(self,weightAfterTraining,biasAfterTraining,weightAfterTrainingOne,biasAfterTrainingOne):
    averageWeightsLayerZero=np.zeros_like(weightAfterTraining[0])


    for i in range(len(weightAfterTraining)):
      averageWeightsLayerZero=np.add(averageWeightsLayerZero,weightAfterTraining[i])

    averageWeightsLayerZero=averageWeightsLayerZero/numberOfClients
    #print(averageWeightsLayerZero)



    
    averageWeightsLayerOne=np.zeros_like(weightAfterTrainingOne[0])

    for i in range(len(weightAfterTrainingOne)):
      averageWeightsLayerOne=np.add(averageWeightsLayerOne,weightAfterTrainingOne[i])
    
    averageWeightsLayerOne=averageWeightsLayerOne/numberOfClients
    #print(averageWeightsLayerOne)


    
    averageBiasLayerZero=np.zeros_like(biasAfterTraining[0])

    for i in range(len(biasAfterTraining)):
      averageBiasLayerZero=np.add(averageBiasLayerZero,biasAfterTraining[i])

    averageBiasLayerZero=averageBiasLayerZero/numberOfClients
    #print(averageBiasLayerZero)
    




    averageBiasLayerOne = np.zeros_like(biasAfterTrainingOne[0])

    for i in range(len(biasAfterTrainingOne)):
      averageBiasLayerOne=np.add(averageBiasLayerOne,biasAfterTrainingOne[i])

    averageBiasLayerOne=averageBiasLayerOne/numberOfClients
    #print(averageBiasLayerOne)

    
    
    #print(weightAfterTraining)
    #print(biasAfterTraining)
    WeightsForServerLayerZero=[]
    WeightsForServerLayerOne=[]
    WeightsForServerLayerZero.append(averageWeightsLayerZero)
    WeightsForServerLayerZero.append(averageBiasLayerZero)
    WeightsForServerLayerOne.append(averageWeightsLayerOne)
    WeightsForServerLayerOne.append(averageBiasLayerOne)
    WeightsForServer=[]
    WeightsForServer.append(WeightsForServerLayerZero)
    WeightsForServer.append(WeightsForServerLayerOne)

    #WeightsForServer=server.calculateAvg(weightAfterTraining,biasAfterTraining,weightAfterTrainingOne,biasAfterTrainingOne)



    
    
    return WeightsForServer

  


In [ ]:
class Client(Model,Main):
  def trainClient(self,numberOfClients,randomWeights,units,per,train_data_copy,train_label_data_copy,dropClient):
      train_data,train_label_data=Client.dataPartition(numberOfClients,train_data_copy,train_label_data_copy,per,dropClient)
      weightAfterTrainingForLayerOne = []
      biasAfterTrainingForLayerOne = []
      weightAfterTrainingOne = []
      biasAfterTrainingOne = []
      
      if(per==2):
        numberOfClients-=1
      for i in range(numberOfClients):
        model=Model.modelDef(units)
        model=Model.ModelSetWeights(model,randomWeights)
        weightsAfterTraining,weightsOne,model=Model.ModelFit(model,train_data[i],train_label_data[i])
        weightAfterTrainingForLayerOne.append(weightsAfterTraining[0])
        biasAfterTrainingForLayerOne.append(weightsAfterTraining[1])
        weightAfterTrainingOne.append(weightsOne[0])
        biasAfterTrainingOne.append(weightsOne[1])
      return weightAfterTrainingForLayerOne,biasAfterTrainingForLayerOne,weightAfterTrainingOne,biasAfterTrainingOne


  def dataPartition(numberOfClients,train_data_copy,train_label_data_copy,per,dropClient):
    import random
    data = 40000
    data_per_user = data // numberOfClients
  
 
    if(per==1):
      train_label_data_copy=Client.dataManipulation(numberOfClients,train_label_data_copy)

    #shuffled_label_data=dataManipulation(clientNumber,percentageOfData,shuffled_label_data)
    train_data = []
    train_label_data=[]
    j=0
    k=data_per_user
    for i in range(numberOfClients):
      if(per==2 and i==dropClient):
        j=j+data_per_user
        k=k+data_per_user
        continue
      train_data.append(train_data_copy[j:k])
      train_label_data.append( train_label_data_copy[j:k])
      j=j+data_per_user
      k=k+data_per_user

    return train_data,train_label_data



  def dataManipulation(numberOfClients,shuffled_label_data):
    clientNumber=int(input("Enter Client Number:"))
    percentage=int(input("Enter Percentage Of Data You want To Manipulate:"))
    dataPerUser=40000//numberOfClients
    j=int(dataPerUser*(clientNumber-1))
    k=int(j+(dataPerUser*percentage)/100)
    #print(j)
    #print(k)
    print("Amount of Data Changed: ",k-j)
    
    for i in range(j,k):
      if(shuffled_label_data[i]==0):
        shuffled_label_data[i]=1;
      else:
        shuffled_label_data[i]=0;

    return shuffled_label_data







In [ ]:
class Finding_Anomaly():

  def kMeansClustering(self,clusterSize,normalizedWeights):
    from sklearn.cluster import KMeans

    # Load your dataset into X
    X = normalizedWeights

    # Define the number of clusters
    #k = 2

    # Create a KMeans object with k clusters
    kmeans = KMeans(n_clusters=clusterSize)

    # Fit the KMeans model on the data
    kmeans.fit(X)

    # Get the cluster assignments for each data point
    labels = kmeans.labels_
    #print(labels)

    # Get the cluster centers
    centers = kmeans.cluster_centers_

    return labels,centers



  def euclideanDistance(self,weights_array):
    weights_mean = np.mean(weights_array, axis=0)
    distances=[]
    for i in range(len(weightAfterTraining)):
      distance_1_2 = np.linalg.norm(weights_mean - weightAfterTraining[i])
      distances.append(distance_1_2)
      print(distance_1_2)

    return distances

  def cosineSimilarity(self,a,b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)


  def manhattanDistance(self,weights_array):
    weights_mean = np.mean(weights_array, axis=0)
    distances = []
    for weight in weights_array:
        distance = np.sum(np.abs(weight - weights_mean))
        distances.append(distance)
        print(distance)
    return distances





In [ ]:
if __name__ == '__main__':
    # Create an instance of the Main class and run the program

    #object of Main Class
    main = Main()
    import numpy as np

    #Object Of Server Class
    server=Server()

    #Define Units
    units=4
    randomWeights=[]

    #Generating Random Weights
    randomWeights=server.randomWeights(units)

    #print(randomWeights)


    numberOfClients=10
     #int(input("Enter The Number Of Clients:"))
    # numberOfClients=
    client=Client()

    # inputs,label_data=main.readingDataFromCSV()
    train_data_copy,test_data,train_label_data_copy,test_label=main.dataSplit()
    
    #Weights After Training
    weightAfterTraining,biasAfterTraining,weightAfterTrainingOne,biasAfterTrainingOne=client.trainClient(numberOfClients,randomWeights,units,0,train_data_copy,train_label_data_copy,0)
    print(np.shape(weightAfterTraining))
    WeightsForServer=server.calculateAvg(weightAfterTraining,biasAfterTraining,weightAfterTrainingOne,biasAfterTrainingOne)

    model=Model()
    model=Model.modelDef(4)
    model=Model.ModelSetWeights(model,WeightsForServer)
    # inputs,label_data=Main.readingDataFromCSV()
    # train_data_copy,test_data,train_label_data_copy,test_label=Main.dataSplit(inputs,label_data)
    from sklearn.metrics import accuracy_score

    # Obtain the predicted outputs of the model
    y_pred = model.predict(test_data)

    #print(y_pred)
    #print(test_label)

    for i in range(len(y_pred)):
      if(y_pred[i]>0.5):
        y_pred[i]=1
      else:
        y_pred[i]=0

    #print(y_pred)

    # Calculate the accuracy score
    accuracy = accuracy_score(test_label, y_pred)

    print("Accuracy of the model Before Changing Data: {:.2f}%".format(accuracy * 100))





    #Forging the data


    weightAfterTraining,biasAfterTraining,weightAfterTrainingOne,biasAfterTrainingOne=client.trainClient(numberOfClients,randomWeights,units,1,train_data_copy,train_label_data_copy,0)
    print(np.shape(weightAfterTraining))
    WeightsForServer1=server.calculateAvg(weightAfterTraining,biasAfterTraining,weightAfterTrainingOne,biasAfterTrainingOne)

    import numpy as np
    weights_array1 = np.array(weightAfterTraining)

    flattendWeightedArray=[]
    for i in range(numberOfClients):
      flattendWeightedArray.append(weights_array1[i].reshape(units*24,))

    mean = np.mean(flattendWeightedArray)
    std = np.std(flattendWeightedArray)

    # Normalize the weights using z-score normalization
    normalizedWeights = (flattendWeightedArray - mean) / std

    #Object Of Finding_Anomaly Class
    anamolyFinder=Finding_Anomaly()

    #Clustering Result
    k=2
    clusters,centers=anamolyFinder.kMeansClustering(k,normalizedWeights)

    print("Result Of Clusters:")
    print(clusters)
    print()

    len0=0
    len1=0
    for i in range(len(clusters)):
      if(clusters[i]==0):
        len0+=1
      else:
        len1+=1

    # print(len0)
    # print(len1)

    if(len1==1):
      for i in range(len(clusters)):
        if(clusters[i]==1):
          print("Anomoly found using Clustering:",i+1)
          break
    elif(len0==1):
      for i in range(len(clusters)):
        if(clusters[i]==0):
          print("Anomoly found using Clustering:",i+1)
          break
    else:
      if(len0==5 and len1==5 or len0==4 and len1==6 or len1==4 and len0==6):
        print("There are NO anomalies")
      else:
        print("Couldn't Find Using Clustering, There may not be any anomalies.")
        print("Can't be sure need to check data to be sure ")

    
    if(len0<len1):
      for i in range(len(clusters)):
        if(clusters[i]==0):
          clusters[i]=0.7
        else:
          clusters[i]=0.3

    if(len0>len1):
      for i in range(len(clusters)):
        if(clusters[i]==0):
          clusters[i]=0.3
        else:
          clusters[i]=0.7
  
    

    #Euclidean Result
    print()
    distances=anamolyFinder.euclideanDistance(weights_array1)
    print("Result Of Euclidean:")
    print(distances)

    max=-23123213
    answer=0
    for i in range(len(distances)):
      if(max<distances[i]):
        max=distances[i]
        answer=i+1

    print("Anomoly found Using Euclidean:",answer)

    print()


    #Manhattan Result
    print()
    distancesMan=anamolyFinder.manhattanDistance(weights_array1)
    print("Result Of Manhattan:")
    print(distancesMan)

    max=-23123213
    answer=0
    for i in range(len(distances)):
      if(max<distancesMan[i]):
        max=distancesMan[i]
        answer=i+1

    print("Anomoly found Using Manhatten:",answer)

    print()

    clusters = np.array(clusters)
    distances = np.array(distances)
    distancesMan = np.array(distancesMan)
    result=(clusters*0.2)+(distances*0.5)+(distancesMan*0.3)

    max=-23123213
    answer=0
    for i in range(len(result)):
      if(max<result[i]):
        max=result[i]
        answer=i+1

    print("Anomoly Detected Using Weighted Average : ",answer)

    #Cosine Similarity Result
    print()
    for i in range(numberOfClients-1):
      for j in range(i+1,numberOfClients):
        similarity=anamolyFinder.cosineSimilarity(flattendWeightedArray[i],flattendWeightedArray[j])
        print("USER:"+str(i)+" USER:"+str(j))
        print(similarity)
        print("!!!!!!!!!!!!!!!")


    print(np.shape(WeightsForServer1))
    print(len(WeightsForServer1))
    model=Model()
    model=Model.modelDef(4)
    model=Model.ModelSetWeights(model,WeightsForServer1)
    from sklearn.metrics import accuracy_score

    # Obtain the predicted outputs of the model
    y_pred = model.predict(test_data)

    for i in range(len(y_pred)):
      if(y_pred[i]>0.5):
        y_pred[i]=1
      else:
        y_pred[i]=0

    #print(y_pred)

    # Calculate the accuracy score
    accuracy = accuracy_score(test_label, y_pred)

    print("Accuracy of the model After Changing Data: {:.2f}%".format(accuracy * 100))
    #weightsAfterTraining,weightsOne=Model.ModelFit(model,train_data[i],train_label_data[i])








    weightAfterTraining,biasAfterTraining,weightAfterTrainingOne,biasAfterTrainingOne=client.trainClient(numberOfClients,randomWeights,units,2,train_data_copy,train_label_data_copy,answer-1)
    WeightsForServerAfterDropping=server.dropAvg(weightAfterTraining,biasAfterTraining,weightAfterTrainingOne,biasAfterTrainingOne,answer)

    model=Model()
    model=Model.modelDef(4)
    model=Model.ModelSetWeights(model,WeightsForServerAfterDropping)
    from sklearn.metrics import accuracy_score

    # Obtain the predicted outputs of the model
    y_pred = model.predict(test_data)
    

    for i in range(len(y_pred)):
      if(y_pred[i]>0.5):
        y_pred[i]=1
      else:
        y_pred[i]=0

    #print(y_pred)

    # Calculate the accuracy score
    accuracy = accuracy_score(test_label, y_pred)

    print("Accuracy of the model After dropping client: {:.2f}%".format(accuracy * 100))
    #weightsAfterTraining,weightsOne=Model.ModelFit(model,train_data[i],train_label_data[i])









    

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(40000,)
[0 1 1 ... 1 0 1]
Epoch 1/4
125/125 [==============================] - 1s 2ms/step - loss: 0.6156
Epoch 2/4
125/125 [==============================] - 0s 2ms/step - loss: 0.5553
Epoch 3/4
125/125 [==============================] - 0s 2ms/step - loss: 0.5427
Epoch 4/4
125/125 [==============================] - 0s 2ms/step - loss: 0.5375
Epoch 1/4
125/125 [==============================] - 1s 2ms/step - loss: 0.6327
Epoch 2/4
125/125 [==============================] - 0s 2ms/step - loss: 0.5816
Epoch 3/4
125/125 [==============================] - 0s 2ms/step - loss: 0.5708
Epoch 4/4
125/125 [==============================] - 0s 2ms/step - loss: 0.5655
Epoch 1/4
125/125 [==============================] - 1s 2ms/step - loss: 0.

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:2007: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


179/179 [==============================] - 0s 2ms/step
Accuracy of the model After Changing Data: 76.11%
Epoch 1/4
125/125 [==============================] - 1s 2ms/step - loss: 0.6152
Epoch 2/4
125/125 [==============================] - 0s 2ms/step - loss: 0.5540
Epoch 3/4
125/125 [==============================] - 0s 2ms/step - loss: 0.5420
Epoch 4/4
125/125 [==============================] - 0s 2ms/step - loss: 0.5375
Epoch 1/4
125/125 [==============================] - 1s 2ms/step - loss: 0.6336
Epoch 2/4
125/125 [==============================] - 0s 2ms/step - loss: 0.5831
Epoch 3/4
125/125 [==============================] - 0s 2ms/step - loss: 0.5715
Epoch 4/4
125/125 [==============================] - 0s 2ms/step - loss: 0.5659
Epoch 1/4
125/125 [==============================] - 2s 2ms/step - loss: 0.6390
Epoch 2/4
125/125 [==============================] - 0s 2ms/step - loss: 0.5886
Epoch 3/4
125/125 [==============================] - 0s 2ms/step - loss: 0.5765
Epoch 4/4
125/1